Grad-CAM

In [ ]:
# === ACCESS DRIVE THROUGH COLAB ==
# from google.colab import drive
# drive.mount('/content/drive')

# # Check if directory exists
# if os.path.exists(ABDOMEN_cesarean_train):
#     print(f"Dataset found at {ABDOMEN_cesarean_train}")
#     print("Listing files in dataset directory:\n")
#     print(os.listdir(ABDOMEN_cesarean_train))  # List files and folders
# else:
#     print(f"Dataset not found at {ABDOMEN_cesarean_train}. Check the path!")

In [13]:
import os
import torch
import torch.nn as nn
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torchvision import transforms, models
from PIL import Image
import torchvision.transforms.functional as TF
import torchvision.utils as vutils

In [14]:
# === Define device ===
device = "cuda" if torch.cuda.is_available() else "cpu"

next cell: recreate architecture to match the one used for training before loading the weights

In [15]:
# === LOAD DENSENET169 MODEL ===
model = models.densenet169(weights=None)
model.features.conv0 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.classifier = nn.Linear(model.classifier.in_features, 2)
model.load_state_dict(torch.load("C:/Users/anale/OneDrive/Documentos/Universidade/TESE/MSc-Thesis/model_paths/abdomen_cv1_densenet169_beat-model.pth", map_location=device))
model.to(device)
model.eval()

C:\Users\anale\AppData\Local\Temp\ipykernel_7260\1095850906.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("C:/Users/anale/OneDrive/Doc

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [16]:
# === Define Grad-CAM ===  
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = None
        self.gradients = None
        self.activations = None
        self.hook_layers(target_layer)

    def hook_layers(self, target_layer): # FP and BP
        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0] # activations' gradients during backward [1, 512, 3, 3]
        def forward_hook(module, input, output):
            self.activations = output # layer4 activation during forward [1, 512, 3, 3]


        # Find the layer by name
        for name, module in self.model.named_modules():
            if name == target_layer:
                self.target_layer = module
                module.register_forward_hook(forward_hook)
                module.register_full_backward_hook(backward_hook)
                break

        if self.target_layer is None:
            raise ValueError(f"Layer {target_layer} not found in model.")


    def generate(self, input_tensor, class_indices=None):
        output = self.model(input_tensor) # forward pass
        if class_indices is None: # if target class is not specified, it uses class prediction (argmax)
            class_indices = torch.argmax(output, dim=1)

        self.model.zero_grad() # Clear any previous gradients
        one_hot = torch.zeros_like(output, device=device)
        for i, class_idx in enumerate(class_indices):
            one_hot[i, class_idx] = 1

        output.backward(gradient=one_hot) #
        self.model.zero_grad() # Ensure no gradients persist after Grad-CAM computation


         # Compute the Grad-CAM heatmap
        weights = torch.nn.functional.adaptive_avg_pool2d(self.gradients, 1) # avgpool - reduces spatial resolution for output_size=1 -> [1, 512, 1, 1]
        cam = torch.sum(weights * self.activations, dim=1, keepdim=True) # cam operation - [1,1,3,3]
        cam = torch.relu(cam)

        # Normalize Grad-CAM heatmap
        cam = cam / (cam.max(dim=2, keepdim=True)[0].max(dim=3, keepdim=True)[0] + 1e-6)
        return cam.cpu().detach().numpy(), class_indices.cpu().numpy()


In [17]:
# === Define Image Transformations === (Ensure grayscale input and correct format)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((80, 80)),  # Match model's input size
    transforms.ToTensor(),
])

In [18]:
# === Extract Ground Truth from image label ===
def get_ground_truth_label(img_name, input_folder):
    if "cesarean" in input_folder.lower():
        return "Cesarean Birth"
    elif "vaginal" in input_folder.lower():
        return "Vaginal Birth"
    elif "Cesarean" in img_name:
        return "Cesarean Birth"
    elif "Vaginal" in img_name:
        return "Vaginal Birth"
    else:
        print(f"Warning: Could not determine ground truth for {img_name}. Assuming 'Cesarean Birth'.")
        return "Cesarean Birth"

In [19]:
# === Prediction Frame ===
def add_colored_frame(image, color, thickness=20):
    height, width = image.shape[:2]
    cv2.rectangle(image, (0, 0), (width - 1, height - 1), color, thickness)
    return image

In [20]:
# === Heatmap Overlay ===
def overlay_heatmap(original_image, heatmap): #heatmap w\ size [1,3,3]
    heatmap = cv2.resize(heatmap[0], (original_image.shape[1], original_image.shape[0])) #heatmap size from [3,3] to [80,80]
    heatmap = np.uint8(255 * heatmap) #adapt to image format needed for color mapping
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET) #apply color map
    original_image = cv2.cvtColor(original_image, cv2.COLOR_GRAY2BGR) # Convert grayscale to 3-channel image
    overlay = cv2.addWeighted(original_image, 0.6, heatmap, 0.4, 0) #blend - 60% of grayscale image , 40% for colored heatmap
    return overlay

In [21]:
# === Apply grad-cam in each image from input folder == Apply Frame == Build Grid == Calculate Mean Correct Heatmap ===
def apply_gradcam_and_save(model, input_folder, output_folder, grid_image_size=(80, 80)):
    os.makedirs(output_folder, exist_ok=True)
    grad_cam = GradCAM(model, 'features.denseblock4.denselayer32')

    image_list = []
    image_names = []
    original_images = []
    correct_heatmaps = []
    incorrect_heatmaps = []

    misclassified_tensors = []
    correctly_classified_tensors = []

    class_names = ["Cesarean Birth", "Vaginal Birth"]

    for img_name in os.listdir(input_folder):
        if not img_name.lower().endswith((".png", ".jpg", ".jpeg")):
            continue

        img_path = os.path.join(input_folder, img_name)

        # Load image
        image = Image.open(img_path)

        # Keep a copy of the original grayscale version for visualization
        original_np = np.array(image.convert("L"))
        original_images.append(original_np)

        # Transform image to tensor: [1, 80, 80] → unsqueeze → [1, 1, 80, 80]
        input_tensor = transform(image).unsqueeze(0).to(device)

        # Grad-CAM
        cam_heatmaps, predicted_classes = grad_cam.generate(input_tensor)
        prediction = predicted_classes[0]
        predicted_label = class_names[prediction]


        # === Ground truth from folder name ===
        ground_truth_label = get_ground_truth_label(img_name, input_folder)
        ground_truth_class = class_names.index(ground_truth_label)

        # === Choose frame color ===
        frame_color = (0, 255, 0)  # green
        if prediction != ground_truth_class:
            frame_color = (0, 0, 255)  # red

        # === Overlay and save ===
        overlay = overlay_heatmap(original_np, cam_heatmaps[0]) # cam shape = [1,3,3]
        overlay_with_frame = add_colored_frame(overlay, frame_color, thickness=20)

        save_path = os.path.join(output_folder, f"heatmap_{predicted_label}_{img_name}")
        cv2.imwrite(save_path, overlay_with_frame)
        print(f"Saved: {save_path} ; Predicted: {predicted_label}")

        # === Add to grid tensor ===
        overlay_rgb = cv2.cvtColor(overlay_with_frame, cv2.COLOR_BGR2RGB) #converts to rgb
        overlay_pil = Image.fromarray(overlay_rgb).resize(grid_image_size) # downsizes the image to fit the grid
        tensor_for_grid = TF.to_tensor(overlay_pil) #converts into a pytorch tensor [3,H,W] and normalizes pixel values

        if prediction == ground_truth_class:
            correctly_classified_tensors.append(tensor_for_grid)
            correct_heatmaps.append(cam_heatmaps[0][0])  # 2D spatial heatmap , cam shape = [3,3]
        else:
            misclassified_tensors.append(tensor_for_grid)
            incorrect_heatmaps.append(cam_heatmaps[0][0])

    # === CREATE GRID ===
    final_sorted_grid = misclassified_tensors + correctly_classified_tensors
    if final_sorted_grid:
        grid = vutils.make_grid(final_sorted_grid, nrow=20, padding=5, normalize=True)
        grid_path = os.path.join(output_folder, "grid_overlay_ordered.jpg")
        vutils.save_image(grid, grid_path)
        print(f"Grid saved at: {grid_path}")

    # === CALCULATE CORRECT MEAN HEATMAP ===
    if correct_heatmaps: #list of NumPy arrays (raw grad-cam maps before resizing or coloring), each w\ shape [3, 3]
        mean_heatmap = np.mean(np.stack(correct_heatmaps), axis=0) # turns list of N heatmaps into 3D array w\ shape [N, 3, 3]
        mean_heatmap -= mean_heatmap.min() # subtracts the minimum value from every pixel so the lowest value is 0
        mean_heatmap /= (mean_heatmap.max() + 1e-8) # divides every pixel by the new maximum value (normalize) ; adds 1e-8 to ensure it wont be divided by 0
        heatmap_uint8 = np.uint8(255 * mean_heatmap) # Converts [0.0 – 1.0] → [0 – 255] format needed for OpenCV
        heatmap_colored = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET) # assigns color to the values

        # Resize to match original image size
        h, w = original_images[0].shape[:2]
        heatmap_resized_blocky = cv2.resize(heatmap_colored, (w, h), interpolation=cv2.INTER_NEAREST) #INTER_CUBIC for smoother transitions ; INTER_NEAREST for blocky look
        heatmap_resized_smooth = cv2.resize(heatmap_colored, (w, h), interpolation=cv2.INTER_CUBIC)


        mean_path_blocky = os.path.join(output_folder, "correct_mean_heatmap_blocky_cv3_head_vaginal_test.jpg")
        cv2.imwrite(mean_path_blocky, heatmap_resized_blocky)
        print(f"Mean heatmap (correct only) saved at: {mean_path_blocky}")

        mean_path_smooth = os.path.join(output_folder, "correct_mean_smooth_cv3_head_vaginal_test.jpg")
        cv2.imwrite(mean_path_smooth, heatmap_resized_smooth)
        print(f"Mean heatmap (correct only) saved at: {mean_path_smooth}")

    # === CALCULATE INCORRECT MEAN HEATMAP ===
    if incorrect_heatmaps:
        mean_heatmap_incorrect = np.mean(np.stack(incorrect_heatmaps), axis=0)
        mean_heatmap_incorrect -= mean_heatmap_incorrect.min()
        mean_heatmap_incorrect /= (mean_heatmap_incorrect.max() + 1e-8)
        heatmap_uint8_incorrect = np.uint8(255 * mean_heatmap_incorrect)
        heatmap_colored_incorrect = cv2.applyColorMap(heatmap_uint8_incorrect, cv2.COLORMAP_JET)

        h, w = original_images[0].shape[:2]
        heatmap_resized_blocky_incorrect = cv2.resize(heatmap_colored_incorrect, (w, h), interpolation=cv2.INTER_NEAREST)
        heatmap_resized_smooth_incorrect = cv2.resize(heatmap_colored_incorrect, (w, h), interpolation=cv2.INTER_CUBIC)

        mean_path_blocky_incorrect = os.path.join(output_folder, "incorrect_mean_heatmap_blocky_cv3_head_vaginal_test.jpg")
        cv2.imwrite(mean_path_blocky_incorrect, heatmap_resized_blocky_incorrect)
        print(f"Mean heatmap (incorrect only) saved at: {mean_path_blocky_incorrect}")

        mean_path_smooth_incorrect = os.path.join(output_folder, "incorrect_mean_smooth_cv3_head_vaginal_test.jpg")
        cv2.imwrite(mean_path_smooth_incorrect, heatmap_resized_smooth_incorrect)
        print(f"Mean heatmap (incorrect only) saved at: {mean_path_smooth_incorrect}")


    return correct_heatmaps, original_images

In [25]:
# Define paths
input_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_cv_1/Abdomen_/test/Vaginal Birth'
output_folder = 'C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/results_GradCAM_cv1dataset/X_ABDOMEN_vaginal_test_cv1_desenet169'

# Run the Grad-CAM processing and save grid
all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)


Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/results_GradCAM_cv1dataset/X_ABDOMEN_vaginal_test_cv1_desenet169\heatmap_Vaginal Birth_PU10000998_abdomen1_201.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/results_GradCAM_cv1dataset/X_ABDOMEN_vaginal_test_cv1_desenet169\heatmap_Vaginal Birth_PU10010674_abdomen1_205.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/results_GradCAM_cv1dataset/X_ABDOMEN_vaginal_test_cv1_desenet169\heatmap_Vaginal Birth_PU10011551_abdomen1_206.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/results_GradCAM_cv1dataset/X_ABDOMEN_vaginal_test_cv1_desenet169\heatmap_Vaginal Birth_PU10014793_abdomen1_208.png ; Predicted: Vaginal Birth
Saved: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/results_GradCAM_cv1dataset/X_ABDOMEN_vaginal_test_cv1_desenet169\heatmap_Vaginal Birth_PU100